### SETTINGS, IMPORT AND LOAD

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)

df = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')

### OVERVIEW DATA ANALYSIS

In [121]:
'''
line = '-'*80
print('HEAD')
print(df.head())
print(line)
print('INFO')
print(df.info())
print(line)
print('DESCRIBE')
print(df.describe().T)
print(line)
print('DESCRIBE NON NUMERIC')
print(df.describe(include=['object', 'bool']))
print(line)
print('COLUMNS')
print(df.columns)
print(line)
'''

"\nline = '-'*80\nprint('HEAD')\nprint(df.head())\nprint(line)\nprint('INFO')\nprint(df.info())\nprint(line)\nprint('DESCRIBE')\nprint(df.describe().T)\nprint(line)\nprint('DESCRIBE NON NUMERIC')\nprint(df.describe(include=['object', 'bool']))\nprint(line)\nprint('COLUMNS')\nprint(df.columns)\nprint(line)\n"

### CHECKING & CORRECTION MISSING VALUES

In [123]:
'''
print('COUNT NULL COLUMNS')
print(df.isnull().sum())       # Count the Null Columns
## print(pd.isnull(df).sum())           # outra forma de fazer
## print(df[df["gender"].isnull()])    # Single Column Is Null
# print(df[df.isnull().any(axis=1)].head())    # dataframe com All Null Columns
'''

'\nprint(\'COUNT NULL COLUMNS\')\nprint(df.isnull().sum())       # Count the Null Columns\n## print(pd.isnull(df).sum())           # outra forma de fazer\n## print(df[df["gender"].isnull()])    # Single Column Is Null\n# print(df[df.isnull().any(axis=1)].head())    # dataframe com All Null Columns\n'

### CHECKING & CORRECTION MISMATCHED VALUES

In [3]:
# df.select_dtypes(include=["float", 'int64'])   # verifica quais colunas são numéricas
# print(df.select_dtypes(include=np.number))   # verifica quais colunas são numéricas. equivalente a anterior

# print(df[df.TotalCharges.str.isspace()]['TotalCharges'])     # identifica campos numéricos com espaço 
# a = df['TotalCharges'].str.findall(r'[^0-9.]')   # verifica caracteres não numéricos e diferente de ponto
# for c, x in enumerate(a):
#     if len(x) > 0:
#         print(c, x)    # mostra linhas com caracteres não numéricos
df['TotalCharges'][df['TotalCharges'] == ' '] = '0'  # verifica as linhas 'TotalCharges' == ' ' e atribui '0'
## df['TotalCharges'] = df['TotalCharges'].replace(r'\s+', np.nan, regex=True) #outra forma: mudar espaços p NaN
## df['TotalCharges']=df['TotalCharges'].fillna(df['TotalCharges'].median())     # uma alternativa é colocar a mediana ao invés de zero

df['TotalCharges'] = pd.to_numeric(df['TotalCharges'])  # transforma a coluna de string para numérico
## df["TotalCharges"] = df["TotalCharges"].astype(float)    # outra forma de converter para float

## type(df.TotalCharges[0])             # verifica o tipo de um determinado campo olhando a primera linha. 
## print(df.TotalCharges)    # verifica se coluna foi convertida para float

#replace values
df["SeniorCitizen"] = df["SeniorCitizen"].replace({1:"Yes",0:"No"})

#replace 'No internet service' to No for the following columns
replace_cols = [ 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
                'TechSupport','StreamingTV', 'StreamingMovies']
for i in replace_cols : 
    df[i]  = df[i].replace({'No internet service' : 'No'})

df['MultipleLines']  = df['MultipleLines'].replace({'No phone service' : 'No'})

### EXPLORATORY DATA ANALYSIS

In [4]:
# print(df.info())

# Separating catagorical and numerical columns
Id_col     = ['customerID']
target_col = ["Churn"]
cat_cols   = df.nunique()[df.nunique() < 6].keys().tolist()
cat_cols   = [x for x in cat_cols if x not in target_col]
num_cols   = [x for x in df.columns if x not in cat_cols + target_col + Id_col]

# Binary columns with 2 values
bin_cols   = df.nunique()[df.nunique() == 2].keys().tolist()
#Columns more than 2 values
multi_cols = [i for i in cat_cols if i not in bin_cols]

# print('CAT', cat_cols, 'NUM', num_cols, 'BIN', bin_cols, 'MULTI', multi_cols)

print('CHURN VALUE COUNTS')
print(df.Churn.value_counts(normalize=True))     # mostra distribuição do target normalizada

CHURN VALUE COUNTS
No    0.73
Yes   0.27
Name: Churn, dtype: float64


### HISTOGRAM - NUMERIC COLUMNS

In [171]:
'''
for c in num_cols:
    plt.figure()
    plt.title(c)
    plt.hist(df[df.Churn == 'Yes'][c], normed=True, bins=20, color='r', alpha=0.5, label='Churn')
    plt.hist(df[df.Churn == 'No'][c], normed=True, bins=20, color='g', alpha=0.5, label='No Churn')
    plt.legend()
    plt.figure()
    plt.title(c)
    plt.hist(df[df.Churn == 'Yes'][c], normed=True, bins=20, color='r', alpha=0.5, label='Churn')
    plt.legend()
    plt.figure()
    plt.title(c)
    plt.hist(df[df.Churn == 'No'][c], normed=True, bins=20, color='g', alpha=0.5, label='No Churn')
    plt.legend()
'''

"\nfor c in num_cols:\n    plt.figure()\n    plt.title(c)\n    plt.hist(df[df.Churn == 'Yes'][c], normed=True, bins=20, color='r', alpha=0.5, label='Churn')\n    plt.hist(df[df.Churn == 'No'][c], normed=True, bins=20, color='g', alpha=0.5, label='No Churn')\n    plt.legend()\n    plt.figure()\n    plt.title(c)\n    plt.hist(df[df.Churn == 'Yes'][c], normed=True, bins=20, color='r', alpha=0.5, label='Churn')\n    plt.legend()\n    plt.figure()\n    plt.title(c)\n    plt.hist(df[df.Churn == 'No'][c], normed=True, bins=20, color='g', alpha=0.5, label='No Churn')\n    plt.legend()\n"

### BAR PLOT

In [41]:
'''
for c in cat_cols:
    # print(df[c].value_counts())
    # print(df[c].value_counts(normalize=True))     # mostra distribuição do target normalizada

    sum_values = df.groupby(['Churn', c]).agg(np.count_nonzero).iloc[:,0]
    yes_values = sum_values.loc['Yes'].values
    no_values = sum_values.loc['No'].values
    xvals = list(sum_values.loc['Yes'].index)
    yes_values_norm = []
    no_values_norm = []
    for x in xvals:
        tot = sum_values.loc[:,x].sum()
        yes_values_norm.append(sum_values.loc['Yes',x] / tot)
        no_values_norm.append(sum_values.loc['No',x] / tot)
        
    # xvals_str = [str(x) for x in xvals]
    plt.figure(figsize=(10,6))
    plt.xlabel(c,  fontsize=12)
    bars_yes = plt.bar(xvals, yes_values, width=0.5, label='Churn Yes', color='r', alpha=0.7)
    bars_no = plt.bar(xvals, no_values, width=0.5, bottom=yes_values, label='Churn No', color='g', alpha=0.7)
    plt.legend()

    # direct label each bar with Y axis values
    for i, bar in enumerate(bars_yes):
        height = bar.get_height()
        width = bar.get_width()
        plt.gca().text(bar.get_x() + width/2, height * (1/2), str(int(height)) + ' , ' + str(int(yes_values_norm[i]*100)) + '%', ha='center', 
        color='black', fontsize=11)

    for i, bar in enumerate(bars_no):
        height = bar.get_height()
        width = bar.get_width()
        plt.gca().text(bar.get_x() + width/2, (height * (1/2)) + yes_values[i], str(int(height)) + ' , ' + str(int(no_values_norm[i]*100)) + '%', 
        ha='center', color='black', fontsize=11)
'''

"\nfor c in cat_cols:\n    # print(df[c].value_counts())\n    # print(df[c].value_counts(normalize=True))     # mostra distribuição do target normalizada\n\n    sum_values = df.groupby(['Churn', c]).agg(np.count_nonzero).iloc[:,0]\n    yes_values = sum_values.loc['Yes'].values\n    no_values = sum_values.loc['No'].values\n    xvals = list(sum_values.loc['Yes'].index)\n    yes_values_norm = []\n    no_values_norm = []\n    for x in xvals:\n        tot = sum_values.loc[:,x].sum()\n        yes_values_norm.append(sum_values.loc['Yes',x] / tot)\n        no_values_norm.append(sum_values.loc['No',x] / tot)\n        \n    # xvals_str = [str(x) for x in xvals]\n    plt.figure(figsize=(10,6))\n    plt.xlabel(c,  fontsize=12)\n    bars_yes = plt.bar(xvals, yes_values, width=0.5, label='Churn Yes', color='r', alpha=0.7)\n    bars_no = plt.bar(xvals, no_values, width=0.5, bottom=yes_values, label='Churn No', color='g', alpha=0.7)\n    plt.legend()\n\n    # direct label each bar with Y axis values\n

### DATA PREPROCESSING

In [5]:
# from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

#Label encoding Binary columns
le = LabelEncoder()
for i in bin_cols :
    df[i] = le.fit_transform(df[i])
    
#Duplicating columns for multi value columns
df = pd.get_dummies(data = df,columns = multi_cols)

#Scaling Numerical columns
std = StandardScaler()
scaled = std.fit_transform(df[num_cols])
scaled = pd.DataFrame(scaled,columns=num_cols)

#dropping original values merging scaled values for numerical columns
df_og = df.copy()
df = df.drop(columns = num_cols,axis = 1)
df = df.merge(scaled,left_index=True,right_index=True,how = "left")

In [43]:
'''

df['Churn'][df['Churn'] == 'No'] = 0     # atriui 0 onde era 'No'
df['Churn'][df['Churn'] == 'Yes'] = 1
# df['Churn'] = df['Churn'].replace({'Yes': 1, 'No': 0})     # forma alternativa em 1 linha


col = 'Dependents'
df[col][df[col] == 'No'] = 0     # atriui 0 onde era 'No'
# df['Churn']=df['Churn'].replace('No',0)    # outra forma de fazer
df[col][df[col] == 'Yes'] = 1

col = 'Partner'
df[col][df[col] == 'No'] = 0     # atriui 0 onde era 'No'
df[col][df[col] == 'Yes'] = 1 

col = 'PaperlessBilling'
df[col][df[col] == 'No'] = 0     # atriui 0 onde era 'No'
df[col][df[col] == 'Yes'] = 1  

le = preprocessing.LabelEncoder()
col = 'TechSupport'
# le.fit(df[col])
df.at[:,col] = le.fit_transform(df[col])
dict_TechSupport = {'No': 0, 'Yes': 2, 'No internet service': 1}

col = 'StreamingTV'
le.fit(df[col])
df.at[:,col] = le.transform(df[col])
dict_TechSupport = {'No': 0, 'Yes': 2, 'No internet service': 1}

col = 'StreamingMovies'
le.fit(df[col])
df.at[:,col] = le.transform(df[col])
dict_TechSupport = {'No': 0, 'Yes': 2, 'No internet service': 1}

col = 'OnlineSecurity'
le.fit(df[col])
df.at[:,col] = le.transform(df[col])
dict_TechSupport = {'No': 0, 'Yes': 2, 'No internet service': 1}

col = 'DeviceProtection'
le.fit(df[col])
df.at[:,col] = le.transform(df[col])
dict_TechSupport = {'No': 0, 'Yes': 2, 'No internet service': 1}

col = 'Contract'
le.fit(df[col])
df.at[:,col] = le.transform(df[col])
dict_Contract = {'Month-to-month': 0, 'Two year': 2, 'One year': 1}

col = 'PaymentMethod'
le.fit(df[col])
df.at[:,col] = le.transform(df[col])
dict_PaymentMethod = {'Electronic check': 2, 'Mailed check': 3, 'Bank transfer (automatic)': 0, 'Credit card (automatic)': 1}

col = 'gender'
le.fit(df[col])
df.at[:,col] = le.transform(df[col])
dict_TechSupport = {'Male': 1, 'Female': 0}

col = 'InternetService'
le.fit(df[col])
df.at[:,col] = le.transform(df[col])
dict_PaymentMethod = {'Fiber optic': 1, 'DSL': 0, 'No': 2}

col = 'OnlineBackup'
le.fit(df[col])
df.at[:,col] = le.transform(df[col])
dict_TechSupport = {'No': 0, 'Yes': 2, 'No internet service': 1}

col = 'PhoneService'
df[col][df[col] == 'No'] = 0     # atriui 0 onde era 'No'
df[col][df[col] == 'Yes'] = 1
'''

"\n\ndf['Churn'][df['Churn'] == 'No'] = 0     # atriui 0 onde era 'No'\ndf['Churn'][df['Churn'] == 'Yes'] = 1\n# df['Churn'] = df['Churn'].replace({'Yes': 1, 'No': 0})     # forma alternativa em 1 linha\n\n\ncol = 'Dependents'\ndf[col][df[col] == 'No'] = 0     # atriui 0 onde era 'No'\n# df['Churn']=df['Churn'].replace('No',0)    # outra forma de fazer\ndf[col][df[col] == 'Yes'] = 1\n\ncol = 'Partner'\ndf[col][df[col] == 'No'] = 0     # atriui 0 onde era 'No'\ndf[col][df[col] == 'Yes'] = 1 \n\ncol = 'PaperlessBilling'\ndf[col][df[col] == 'No'] = 0     # atriui 0 onde era 'No'\ndf[col][df[col] == 'Yes'] = 1  \n\nle = preprocessing.LabelEncoder()\ncol = 'TechSupport'\n# le.fit(df[col])\ndf.at[:,col] = le.fit_transform(df[col])\ndict_TechSupport = {'No': 0, 'Yes': 2, 'No internet service': 1}\n\ncol = 'StreamingTV'\nle.fit(df[col])\ndf.at[:,col] = le.transform(df[col])\ndict_TechSupport = {'No': 0, 'Yes': 2, 'No internet service': 1}\n\ncol = 'StreamingMovies'\nle.fit(df[col])\ndf.at[:,co

### FEATURE SELECTION

In [6]:
# df.columns
# Index(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
#       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
#       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn']

# features = ['Contract', 'MonthlyCharges', 'tenure', 'TotalCharges', 'OnlineSecurity', 'TechSupport', 'InternetService',  'PaymentMethod',  'SeniorCitizen', 'Dependents', 'Partner', 'OnlineBackup', 'DeviceProtection', 'PaperlessBilling']
features = [i for i in df.columns if i not in Id_col + target_col]
X = df[features]
y = df['Churn']
# y = y.astype('int')

### Synthetic Minority Oversampling TEchnique (SMOTE)

In [9]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

smote_X = df[features]
smote_Y = df[target_col]

#Split train and test data
smote_train_X,smote_test_X,smote_train_Y,smote_test_Y = train_test_split(smote_X,smote_Y, test_size = .25 , random_state = 111)

#oversampling minority class using smote
os = SMOTE(random_state = 0)
os_smote_X,os_smote_Y = os.fit_sample(smote_train_X,smote_train_Y)
os_smote_X = pd.DataFrame(data = os_smote_X,columns=features)
os_smote_Y = pd.DataFrame(data = os_smote_Y,columns=target_col)

### Recursive Feature Elimination

In [12]:
from sklearn.feature_selection import RFE
from sklearn.ensemble import GradientBoostingClassifier

clf_base = GradientBoostingClassifier()

rfe = RFE(clf_base,10)
rfe = rfe.fit(os_smote_X,os_smote_Y.values.ravel())

rfe.support_
rfe.ranking_

#identified columns Recursive Feature Elimination
idc_rfe = pd.DataFrame({"rfe_support" :rfe.support_,
                       "columns" : [i for i in df.columns if i not in Id_col + target_col],
                       "ranking" : rfe.ranking_,
                      })
features = idc_rfe[idc_rfe["rfe_support"] == True]["columns"].tolist()
X = df[features]

### ONE HOT ENCODING

In [50]:
'''
OHE = False
if OHE:
    col = 'Contract'
    df_ohe = df[features]
    ohe = OneHotEncoder(sparse=False) # categorical_features = boolean...
    # for col in features_ohe:
    le.fit(df[col])
    col2 = col + '_e'
    df.at[:,col2] = le.transform(df[col])
    colunas = list(df.groupby([col, col2]).max().index.levels[0])         

    X_ohe = ohe.fit_transform(df[col2].values.reshape(-1, 1)) # It returns an numpy array                
    new = pd.DataFrame(X_ohe, columns=colunas, index=df.index)
    df_ohe = pd.merge(new, df_ohe, how='inner', left_index=True, right_index=True)
    # df = df_ohe.copy()
    df = df_ohe.drop(col, axis=1)
else:
    df = df[features]
'''

"\nOHE = False\nif OHE:\n    col = 'Contract'\n    df_ohe = df[features]\n    ohe = OneHotEncoder(sparse=False) # categorical_features = boolean...\n    # for col in features_ohe:\n    le.fit(df[col])\n    col2 = col + '_e'\n    df.at[:,col2] = le.transform(df[col])\n    colunas = list(df.groupby([col, col2]).max().index.levels[0])         \n\n    X_ohe = ohe.fit_transform(df[col2].values.reshape(-1, 1)) # It returns an numpy array                \n    new = pd.DataFrame(X_ohe, columns=colunas, index=df.index)\n    df_ohe = pd.merge(new, df_ohe, how='inner', left_index=True, right_index=True)\n    # df = df_ohe.copy()\n    df = df_ohe.drop(col, axis=1)\nelse:\n    df = df[features]\n"

### MODELING

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
# from sklearn.preprocessing import MinMaxScaler

# norm = False
# clf = RandomForestClassifier(n_estimators=100, max_features=5, n_jobs = -1, random_state = 0)  # n_estimators=600, 
# clf = RandomForestClassifier(criterion='entropy',n_estimators=1000,max_depth=100,oob_score=True,random_state=42)
clf = GradientBoostingClassifier(n_estimators=100, learning_rate = 0.1, max_depth = 4, random_state = 0)
# clf = KNeighborsClassifier(n_neighbors = 4)
# clf = LogisticRegression(C=0.1)

# norm = True
# clf = MLPClassifier(hidden_layer_sizes = [100, 100], solver='adam',  activation = 'tanh', alpha=0.00001, random_state = 0)   
   
# Fit
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)
# if norm:
#     scaler = MinMaxScaler()
#     X_train = scaler.fit_transform(X_train)
#     X_test = scaler.transform(X_test)

clf.fit(X_train, y_train)

y_proba_lr = clf.predict_proba(X_test)
fpr_lr, tpr_lr, _ = roc_curve(y_test, y_proba_lr[:,1])
roc_auc_lr = auc(fpr_lr, tpr_lr)

y_predicted = clf.predict(X_test)
clr = classification_report(y_test, y_predicted, target_names = ['0', '1'])

### RESULTS

In [14]:
print('score train:', clf.score(X_train, y_train))
print('score test:', clf.score(X_test, y_test))
print('AUC', roc_auc_lr)
print(confusion_matrix(y_test, y_predicted))
print(clr)

score train: 0.8436198409693298
score test: 0.7768313458262351
AUC 0.8244358324985773
[[1131  167]
 [ 226  237]]
              precision    recall  f1-score   support

           0       0.83      0.87      0.85      1298
           1       0.59      0.51      0.55       463

    accuracy                           0.78      1761
   macro avg       0.71      0.69      0.70      1761
weighted avg       0.77      0.78      0.77      1761



In [15]:
feature_importances1 = pd.DataFrame(clf.feature_importances_,index = X_train.columns,columns=['importance']).sort_values('importance',ascending=False)
pd.set_option('display.float_format', '{:.6f}'.format)
feature_importances1

,importance
Contract_Month-to-month,0.388247
tenure,0.158231
TotalCharges,0.136396
MonthlyCharges,0.135372
InternetService_Fiber optic,0.098715
PaymentMethod_Electronic check,0.040301
InternetService_No,0.017559
OnlineSecurity,0.016041
Contract_Two year,0.004741
Contract_One year,0.004396


In [29]:
# X.mean()
# X_test
# scaled = std.fit_transform(df[num_cols])
# scaled = pd.DataFrame(scaled,columns=num_cols)
X_sim = pd.DataFrame(columns=X_test.columns)
X_sim.at[0, 'OnlineSecurity'] = 0
X_sim.at[0, 'InternetService_Fiber optic'] = 1
X_sim.at[0, 'InternetService_No'] = 0
X_sim.at[0, 'Contract_Month-to-month'] = 1
X_sim.at[0, 'Contract_One year'] = 0
X_sim.at[0, 'Contract_Two year'] = 0
X_sim.at[0, 'PaymentMethod_Electronic check'] = 1
X_sim.at[0, 'tenure'] = -1
X_sim.at[0, 'MonthlyCharges'] = -1
X_sim.at[0, 'TotalCharges'] = -1
# X_sim
y_predicted_sim = clf.predict(X_sim)
y_predicted_sim

array([1])

In [25]:
X_sim.columns

Index(['OnlineSecurity', 'InternetService_Fiber optic', 'InternetService_No',
       'Contract_Month-to-month', 'Contract_One year', 'Contract_Two year',
       'PaymentMethod_Electronic check', 'tenure', 'MonthlyCharges',
       'TotalCharges'],
      dtype='object')

### Melhorias
- organizar e deixar modelo

- https://www.kaggle.com/pavanraj159/telecom-customer-churn-prediction
- Recursive Feature Elimination (BOM)
- PCA - principal components
- Threshold Plot 
- Univariate Selection

- cross validation (avaliar - tenho dúvidas)
- scatter plot matrix for numerical columns (interesante mas não mandatório)
- radar chart?? (não prioritário)